In [65]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
import numpy as np

db = mysql.connector.connect(host = "localhost", username= "root", password = "Kdpms@171", database = "ecommerce")
cur = db.cursor()

# Calculate the moving average of order values for each customer over their order history.

In [66]:
query10 = """select customer_id, order_purchase_timestamp, payment, 
avg(payment) over(partition by customer_id order by order_purchase_timestamp rows between 2 preceding and current row) as moving_avg 
from (select orders.customer_id, orders.order_purchase_timestamp, payments.payment_value as payment from payments 
join orders on payments.order_id = orders.order_id) as a"""
cur.execute(query10)
data10 = cur.fetchall()
df10 = pd.DataFrame(data10, columns = ["customer_id", "time_staamp", "payment", "moving_average"])
df10.head(15)


,customer_id,time_staamp,payment,moving_average
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
1,000161a058600d5901f007fab4c27140,2017-07-16 09:40:32,67.41,67.410004
2,0001fd6190edaaf884bcaf3d49edf079,2017-02-28 11:06:43,195.42,195.419998
3,0002414f95344307404f0ace7a26f1d5,2017-08-16 13:09:20,179.35,179.350006
4,000379cdec625522490c315e70c7a9fb,2018-04-02 13:42:17,107.01,107.010002
5,0004164d20a9e969af783496f3408652,2017-04-12 08:35:12,71.80,71.800003
6,000419c5494106c306a97b5635748086,2018-03-02 17:47:40,49.40,49.400002
7,00046a560d407e99b969756e0b10f282,2017-12-18 11:08:30,166.59,166.589996
8,00050bf6e01e69d5c0fd612f1bcfb69c,2017-09-17 16:04:44,85.23,85.230003
9,000598caf2ef4117407665ac33275130,2018-08-11 12:14:35,1255.71,1255.709961


# Calculate the cumulative sales per month for each year.

In [67]:
query11 = """select years, months, payment, sum(payment) over(order by years, months) cumulative_sales from (select year(orders.order_purchase_timestamp) as years, month(orders.order_purchase_timestamp) as months, round(sum(payments.payment_value), 2) as payment from orders 
 join payments on payments.order_id = orders.order_id group by years, months order by years, months) as a"""
cur.execute(query11)
data11 = cur.fetchall()
df11 = pd.DataFrame(data11, columns = ["year", "months", "sales", "cumulative_sales"])
df11

,year,months,sales,cumulative_sales
0,2016,9,252.24,252.24
1,2016,10,59090.48,59342.72
2,2016,12,19.62,59362.34
3,2017,1,138488.04,197850.38
4,2017,2,291908.01,489758.39
5,2017,3,449863.60,939621.99
6,2017,4,417788.03,1357410.02
7,2017,5,592918.82,1950328.84
8,2017,6,511276.38,2461605.22
9,2017,7,592382.92,3053988.14


# Calculate the year-over-year growth rate of total sales.

In [68]:
query12 = """with a as(select year(orders.order_purchase_timestamp) as years, round(sum(payments.payment_value), 2) as payment from orders 
 join payments on payments.order_id = orders.order_id group by years order by years)
 select years, ((payment - lag(payment, 1) over(order by years))/lag(payment, 1) over(order by years))*100 previous_sales from a"""
cur.execute(query12)
data12 = cur.fetchall()
df12 = pd.DataFrame(data12, columns = ["year", "YOY_growth"])
df12

,year,YOY_growth
0,2016,NaN
1,2017,12112.703761
2,2018,20.000924


# Calculate the retention rate of customers, defined as the percentage of customers who make another purchase within 6 months of their first purchase.

In [69]:
query13 = """with a as (select customers.customer_id, min(orders.order_purchase_timestamp) first_order from customers 
join orders on orders.customer_id = customers.customer_id 
group by customers.customer_id), 
b as (select a.customer_id, count(distinct orders.order_purchase_timestamp) from a 
join orders on orders.customer_id = a.customer_id 
and orders.order_purchase_timestamp > first_order 
and orders.order_purchase_timestamp < date_add(first_order, interval 6 month) 
group by a.customer_id)
select 100*(count(distinct a.customer_id)/count(distinct b.customer_id)) 
from a left join b on a.customer_id = b.customer_id;"""
cur.execute(query13)
data13 = cur.fetchall()
data13


[(None,)]

# Identify the top 3 customers who spent the most money in each year.

In [70]:
query14 = """select years, customer_id, payment, d_rank from (select year(orders.order_purchase_timestamp) years, orders.customer_id customer_id, 
sum(payments.payment_value) payment, 
dense_rank() over(partition by year(orders.order_purchase_timestamp) order by sum(payments.payment_value) desc) d_rank from orders 
join payments on payments.order_id = orders.order_id 
group by year(orders.order_purchase_timestamp), orders.customer_id) as temp where d_rank <= 3 ;"""
cur.execute(query14)
data14 = cur.fetchall()
df14 = pd.DataFrame(data14, columns = ["year", "customer_id", "payment", "d_rank"])
df14

,year,customer_id,payment,d_rank
0,2016,a9dc96b027d1252bbac0a9b72d837fc6,1423.550049,1
1,2016,1d34ed25963d5aae4cf3d7f3a4cda173,1400.739990,2
2,2016,4a06381959b6670756de02e07b83815f,1227.780029,3
3,2017,1617b1357756262bfa56ab541c47bc16,13664.080078,1
4,2017,c6e2731c5b391845f6800c97401a43a9,6929.310059,2
5,2017,3fd6777bbce08a352fddd04e4a7cc8f6,6726.660156,3
6,2018,ec5b2ba62e574342386871631fafd3fc,7274.879883,1
7,2018,f48d464a0baaea338cb25f816991ab1f,6922.209961,2
8,2018,e0a2412720e9ea4f26c1ac985f6a7358,4809.439941,3
